# Aula 19: ACID

# Notebook B

### Ponto 1

In [1]:
import mysql.connector

from functools import partial


def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host="localhost",
        user="megadados",
        password="megadados",
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection2, db2 = get_connection_helper("tranqueira")

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

## `SERIALIZABLE`

### Ponto 2

Vamos mudar o nível de isolamento aqui também para `SERIALIZABLE`:

In [2]:
db2('SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE')
db2('START TRANSACTION')

Executando query:
Executando query:


<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 3

Rode esta célula. Você vai ver que o notebook vai ficar parado aqui, esperando o resultado da query. Se você esperar bastante tempo (1 ou 2 minutos) você verá a seguinte mensagem:

```DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction```

In [4]:
db2('INSERT INTO perigo(Nome) VALUES ("Astral")')
db2('SELECT * FROM perigo')

Executando query:
Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Mental')
(5, 'Econômico')
(6, 'Astral')
(7, 'Astral')


O que aconteceu?

- O notebook A adquiriu a trava (lock) da tabela `perigo`;
- O notebook B tentou adquirir essa trava para executar a query;
- Para evitar deadlock o MySQL implementou um mecanismo de timeout na espera pela aquisição da trava.

Agora rode de novo esta célula, mas volte rapidamente para o notebook A e siga as instruções.

<img src="pare.png" width=150px/>

---

Observe que ao fechar a transação em A, liberamos a trava da tabela `perigo` e permitimos que a transação de B continue a ocorrer. 

Não prossiga. Volte para o notebook A e continue a atividade. Você retornará a este ponto aqui em breve.

<img src="pare.png" width=150px/>

---

Vamos encerrar a transação com o cancelamento da operação de inserção:

In [5]:
db2('ROLLBACK')

Executando query:


<img src="pare.png" width=150px/>

Volte para o notebook A.

## `REPEATABLE READ`

### Ponto 4

Vamos mudar o nível de isolamento aqui também:

In [6]:
db2('SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;')

Executando query:


<img src="pare.png" width=150px/>

Volte para o notebook A.

### Ponto 5

In [7]:
db2('START TRANSACTION')

Executando query:


<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 6

Rode a célula abaixo:

In [8]:
db2('INSERT INTO perigo(Nome) VALUES ("Ortopédico")')
db2('SELECT * FROM perigo')

Executando query:
Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Mental')
(5, 'Econômico')
(9, 'Ortopédico')


Observe que, diferente da situação anterior, não houve travamento! 

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 7

Agora confirme a transação:

In [9]:
db2('COMMIT')

Executando query:


Observe que a mudança da tabela foi registrada:

In [10]:
db2('SELECT * FROM perigo')

Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Mental')
(5, 'Econômico')
(9, 'Ortopédico')


<img src="pare.png" width=150px/>

Volte para o notebook A.

### Ponto 8

Vamos abrir uma nova transação e tentar mudar o nome 'Mental' para 'Psicológico':

In [11]:
db2('START TRANSACTION')

Executando query:


In [12]:
db2('UPDATE perigo SET Nome = "Psicológico" WHERE Nome = "Mental"')

Executando query:


In [13]:
db2('SELECT * FROM perigo')

Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Psicológico')
(5, 'Econômico')
(9, 'Ortopédico')


<img src="pare.png" width=150px/>

Volte para o notebook A.

In [14]:
db2('COMMIT')

Executando query:


### Ponto 9

Feche a conexão e volte para o notebook A.

In [15]:
connection2.close()